In [1]:
# what kind of files do we have?
from glob import glob
from pathlib import Path
from collections import Counter

GLOB = '/kaggle/input/chessman-image-dataset/Chessman-image-dataset/Chess/*/*'
suffixes = Counter([Path(input_file).suffix for input_file in glob(pathname=GLOB)])

print(suffixes)

Counter({'.jpg': 465, '.png': 69, '.JPG': 10, '.jpeg': 7, '.fcgi': 2, '.php': 1, '.webp': 1, '.gif': 1})


We have some files that are not images and we need to be sure to ignore them.

Let's build a DataFrame of tags and file names.

In [2]:
import pandas as pd
from pathlib import Path

SUFFIXES = {'.jpg', '.png', '.JPG', '.jpeg', '.gif'}
data = []
for input_file in glob(pathname=GLOB):
    path = Path(input_file)
    if path.suffix in SUFFIXES:
        data.append(pd.Series(data={'tag': str(path.parents[0]).split('/')[-1], 'name': input_file}))
df = pd.DataFrame(data=data)

In [3]:
df['tag'].value_counts()

tag
Pawn      106
Knight    105
Rook      102
Bishop     87
Queen      77
King       75
Name: count, dtype: int64

In [4]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

device = torch.device('cpu')
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1).to(device=device)
model.eval()

layer = model._modules.get('avgpool')

layer_output_size = 512

scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

df['data'] = df['name'].apply(func=lambda x: Image.open(fp=x, mode='r').convert('RGB'))
model_input = [normalize(to_tensor(scaler(item))) for item in df['data'].tolist()]
images = torch.stack(model_input).to(device)
df['value'] = [item.detach().numpy() for item in model(images)]
df = df.drop(columns=['data'])


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 53.2MB/s]
